In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
# read in data by borough and year, add year column, select target columns,
# concatenate by borough then concatenate for whole city with new index

boroughs = ["Bronx", "Brooklyn", "Manhattan", "Queens", "StatenIsland"]
city_df = []

for borough in boroughs:
    borough_df = []
    for Year in range(2003,2019):
        file_name= f"{Year}_{borough}.csv"
        file_path = Path(borough,borough,file_name)
        year_frame = pd.read_csv(file_path, index_col=False)
        year_frame = year_frame.assign(Year=Year)
        borough_df.append(year_frame)
    borough_df = pd.concat(borough_df)
    borough_df = borough_df[["Borough", "Neighborhood", "Building Class Category", "Address", "Apartment Number", 
                            "Zip Code", "Residential Units", 
                            "Land Square Feet", "Gross Square Feet", 
                            "Year", "Sale Price", "Sale Date"]]
    borough_df["Borough"] = borough
    city_df.append(borough_df)
    
city_df = pd.concat(city_df,ignore_index=True)

city_df.columns = city_df.columns.str.replace(' ','_')

city_df.head()


,Borough,Neighborhood,Building_Class_Category,Address,Apartment_Number,Zip_Code,Residential_Units,Land_Square_Feet,Gross_Square_Feet,Year,Sale_Price,Sale_Date
0,Bronx,BATHGATE,01 ONE FAMILY HOMES,1848 WASHINGTON AVENUE,,10457.0,1.0,2943.0,2128.0,2003,130000.0,2003-05-27 00:00:00
1,Bronx,BATHGATE,01 ONE FAMILY HOMES,412 EAST 179 STREET,,10457.0,1.0,1842.0,2048.0,2003,204000.0,2003-04-07 00:00:00
2,Bronx,BATHGATE,01 ONE FAMILY HOMES,410 EAST 182 STREET,,10457.0,1.0,1330.0,1460.0,2003,235000.0,2003-07-24 00:00:00
3,Bronx,BATHGATE,01 ONE FAMILY HOMES,441 EAST 178 STREET,,10457.0,1.0,1287.0,2378.0,2003,125500.0,2003-05-02 00:00:00
4,Bronx,BATHGATE,01 ONE FAMILY HOMES,461 EAST 178 STREET,,10457.0,1.0,1782.0,1548.0,2003,215000.0,2003-05-14 00:00:00


In [4]:
city_df.Building_Class_Category.unique()

array(['01  ONE FAMILY HOMES                        ',
       '02  TWO FAMILY HOMES                        ',
       '03  THREE FAMILY HOMES                      ',
       '05  TAX CLASS 1 VACANT LAND                 ',
       '07  RENTALS - WALKUP APARTMENTS             ',
       '10  COOPS - ELEVATOR APARTMENTS             ',
       '14  RENTALS - 4-10 UNIT                     ',
       '21  OFFICE BUILDINGS                        ',
       '22  STORE BUILDINGS                         ',
       '27  FACTORIES                               ',
       '29  COMMERCIAL GARAGES                      ',
       '30  WAREHOUSES                              ',
       '31  COMMERCIAL VACANT LAND                  ',
       '37  RELIGIOUS FACILITIES                    ',
       '06  TAX CLASS 1 - OTHER                     ',
       '32  HOSPITAL AND HEALTH FACILITIES          ',
       '38  ASYLUMS AND HOMES                       ',
       '41  TAX CLASS 4 - OTHER                     ',
       '08

In [5]:
# check if Residential_Units filters out non residential building class categories

# codes of unique building categories for whole df
total_cat = city_df.Building_Class_Category.str[:2].unique().tolist()

# codes of unique building categories where Residential_Units > 0
res_filtered = city_df[(city_df.Residential_Units > 0)]
res_filtered = res_filtered.Building_Class_Category.str[:2].unique().tolist()

# list of codes in total_cat not in res_filtered
res_class = [x for x in total_cat if x not in res_filtered]
print(res_class)

['18', '24', '47', '42', nan, '49', '45']


In [6]:
# drop Residential_Units Column
city_df.drop('Residential_Units', axis=1, inplace=True)

In [7]:
# drop rows with sqf == 0, price < 1000
city_df = city_df[(city_df.Land_Square_Feet > 0) & (city_df.Gross_Square_Feet > 0) & (city_df.Sale_Price > 1000)]

In [8]:
# filter for target residential_classes
residential_classes = ["01", "02", "03", "04", "06", "07", "08", "09", "10", "11", 
                         "12", "13", "14", "15", "16", "17", "23", "45", "49"]
city_df = city_df[city_df["Building_Class_Category"].str[:2].isin(residential_classes)]

In [9]:
city_df.Building_Class_Category.unique()

array(['01  ONE FAMILY HOMES                        ',
       '02  TWO FAMILY HOMES                        ',
       '03  THREE FAMILY HOMES                      ',
       '07  RENTALS - WALKUP APARTMENTS             ',
       '14  RENTALS - 4-10 UNIT                     ',
       '08  RENTALS - ELEVATOR APARTMENTS           ',
       '09  COOPS - WALKUP APARTMENTS               ',
       '06  TAX CLASS 1 - OTHER                     ',
       '10  COOPS - ELEVATOR APARTMENTS             ',
       '23  LOFT BUILDINGS                          ',
       '11A CONDO-RENTALS                           ',
       '17  CONDOPS                                 ',
       '01  ONE FAMILY DWELLINGS                    ',
       '02  TWO FAMILY DWELLINGS                    ',
       '03  THREE FAMILY DWELLINGS                  ',
       '01 ONE FAMILY DWELLINGS                    ',
       '02 TWO FAMILY DWELLINGS                    ',
       '03 THREE FAMILY DWELLINGS                  ',
       '07 RE

In [10]:
# dataframe with upper and lower quantile for each borough and year
city_df["$sqf"] = city_df["Sale_Price"]/city_df["Gross_Square_Feet"]
lower_quant = city_df.groupby(["Borough", "Year"])["$sqf"].quantile(0.05)
upper_quant = city_df.groupby(["Borough", "Year"])["$sqf"].quantile(0.95)
quant_df = pd.concat([lower_quant, upper_quant], join="outer",axis=1, keys=["lower", "upper"])
quant_df.head()

lower       upper
Borough Year                       
Bronx   2003  37.943262  241.536458
        2004  53.669085  290.591688
        2005  60.522029  329.994289
        2006  71.219426  353.150329
        2007  60.995753  365.035366

In [11]:
# Adding columns with upper and lower $/sqf quantiles 
city_df["Lower_Quant"] = city_df.groupby(["Borough", "Year"])["$sqf"].transform(lambda x: x.quantile(0.05))
city_df["Upper_Quant"] = city_df.groupby(["Borough", "Year"])["$sqf"].transform(lambda x: x.quantile(0.95))
city_df.shape

(480509, 14)

In [12]:
# properties filtered out by quantiles
rejected = city_df[(city_df["$sqf"]<city_df["Lower_Quant"]) | (city_df["$sqf"]>city_df["Upper_Quant"])]
rejected.head()

,Borough,Neighborhood,Building_Class_Category,Address,Apartment_Number,Zip_Code,Land_Square_Feet,Gross_Square_Feet,Year,Sale_Price,Sale_Date,$sqf,Lower_Quant,Upper_Quant
66,Bronx,BATHGATE,03 THREE FAMILY HOMES,2243 BATHGATE AVENUE,,10457.0,1562.0,4224.0,2003,125000.0,2003-06-02 00:00:00,29.592803,37.943262,241.536458
67,Bronx,BATHGATE,03 THREE FAMILY HOMES,2243 BATHGATE AVENUE,,10457.0,1562.0,4224.0,2003,35000.0,2003-06-02 00:00:00,8.285985,37.943262,241.536458
146,Bronx,BAYCHESTER,01 ONE FAMILY HOMES,1124 EAST 213 STREET,,10469.0,2145.0,1920.0,2003,11000.0,2003-11-17 00:00:00,5.729167,37.943262,241.536458
161,Bronx,BAYCHESTER,01 ONE FAMILY HOMES,3477 MICKLE AVENUE,,10469.0,1918.0,1024.0,2003,259500.0,2003-04-30 00:00:00,253.417969,37.943262,241.536458
164,Bronx,BAYCHESTER,01 ONE FAMILY HOMES,3466 MICKLE AVENUE,,10469.0,1879.0,1900.0,2003,25000.0,2003-08-28 00:00:00,13.157895,37.943262,241.536458


In [13]:
# Select values that fall between quantiles
city_df = city_df[(city_df["$sqf"]>city_df["Lower_Quant"]) & (city_df["$sqf"]<city_df["Upper_Quant"])]
city_df.shape

(432373, 14)

In [14]:
# of duplicated_addresses
duplicated_address = city_df["Address"][(city_df["Address"].duplicated()==True)]
duplicated_address.shape

(76502,)

In [15]:
# number of apartments can't fill duplicated addresses
apt_df = city_df["Apartment_Number"].unique()
apt_df.shape

(1325,)

In [16]:
city_df.drop(["Apartment_Number"], axis=1)

,Borough,Neighborhood,Building_Class_Category,Address,Zip_Code,Land_Square_Feet,Gross_Square_Feet,Year,Sale_Price,Sale_Date,$sqf,Lower_Quant,Upper_Quant
0,Bronx,BATHGATE,01 ONE FAMILY HOMES,1848 WASHINGTON AVENUE,10457.0,2943.0,2128.0,2003,130000.0,2003-05-27 00:00:00,61.090226,37.943262,241.536458
1,Bronx,BATHGATE,01 ONE FAMILY HOMES,412 EAST 179 STREET,10457.0,1842.0,2048.0,2003,204000.0,2003-04-07 00:00:00,99.609375,37.943262,241.536458
2,Bronx,BATHGATE,01 ONE FAMILY HOMES,410 EAST 182 STREET,10457.0,1330.0,1460.0,2003,235000.0,2003-07-24 00:00:00,160.958904,37.943262,241.536458
3,Bronx,BATHGATE,01 ONE FAMILY HOMES,441 EAST 178 STREET,10457.0,1287.0,2378.0,2003,125500.0,2003-05-02 00:00:00,52.775442,37.943262,241.536458
4,Bronx,BATHGATE,01 ONE FAMILY HOMES,461 EAST 178 STREET,10457.0,1782.0,1548.0,2003,215000.0,2003-05-14 00:00:00,138.888889,37.943262,241.536458
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1523242,StatenIsland,WOODROW,02 TWO FAMILY DWELLINGS,20 POPLAR LANE,10309.0,2568.0,2700.0,2018,640000.0,2018-05-14 00:00:00,237.037037,157.701466,567.139001
1523244,StatenIsland,WOODROW,02 TWO FAMILY DWELLINGS,35 SALAMANDER COURT,10309.0,2440.0,1300.0,2018,630000.0,2018-04-27 00:00:00,484.615385,157.701466,567.139001
1523247,StatenIsland,WOODROW,02 TWO FAMILY DWELLINGS,64 DARNELL LANE,10309.0,3180.0,1300.0,2018,675000.0,2018-12-28 00:00:00,519.230769,157.701466,567.139001
1523248,StatenIsland,WOODROW,02 TWO FAMILY DWELLINGS,16 DARNELL LANE,10309.0,4431.0,1300.0,2018,630000.0,2018-07-24 00:00:00,484.615385,157.701466,567.139001


In [17]:
# checking for null values
na = city_df.set_index(["Borough"]).isna().sum(level=0)
na.head()

,Neighborhood,Building_Class_Category,Address,Apartment_Number,Zip_Code,Land_Square_Feet,Gross_Square_Feet,Year,Sale_Price,Sale_Date,$sqf,Lower_Quant,Upper_Quant
Borough,,,,,,,,,,,,,
Bronx,0,0,0,3216,0,0,0,0,0,0,0,0,0
Brooklyn,0,0,0,6685,0,0,0,0,0,0,0,0,0
Manhattan,0,0,0,562,0,0,0,0,0,0,0,0,0
Queens,0,0,0,9976,0,0,0,0,0,0,0,0,0
StatenIsland,0,0,0,4851,0,0,0,0,0,0,0,0,0


In [18]:

city_df.reset_index(inplace=True)

In [23]:
address_df = city_df[["Address", "Building_Class_Category", "Borough", "Neighborhood", 
                     "Zip_Code"]]
sale_df = city_df[["Gross_Square_Feet", "Sale_Price", "Sale_Date", "Year"]]

In [26]:
sale_df.to_csv("NY_Propety_Sales_Table.csv")
address_df.to_csv("NY_Property_Address_Table.csv")

In [27]:
rejected.to_csv("Price_Outliers.csv")